# LGMVIP - Data Science Intern, April-2022

## AUTHOR Name :- Sachinkumar Ganesh Navgale

## TASK 9 (Advanced Level Task) Handwritten equation solver using CNN 

Mathematical equation solver using character and symbol recognition using image processing and CNN.

#### Dataset link : https://www.kaggle.com/xainano/handwrittenmathsymbols

## Importing Libraries

In [ ]:
import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.utils import shuffle

In [ ]:
# def load_images_from_folder(folder):
#     train_data=[]
#     for filename in os.listdir(folder):
#         img = cv2.imread(os.path.join(folder,filename),cv2.IMREAD_GRAYSCALE)
#         img=~img        
#         if img is not None:
#             ret,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)

#             ctrs,ret=cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)#DUNG THU CHAIN_APPROX_SIMPLE
#             cnt=sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
#             w=int(45)
#             h=int(45)
#             maxi=0
#             for c in cnt:
#                 x,y,w,h=cv2.boundingRect(c)
#                 maxi=max(w*h,maxi)
#                 if maxi==w*h:
#                     x_max=x
#                     y_max=y
#                     w_max=w
#                     h_max=h
#             im_crop= thresh[y_max:y_max+h_max+10, x_max:x_max+w_max+10]
#             im_resize = cv2.resize(im_crop,(45,45))
#             im_resize=np.reshape(im_resize,(45,45,1))
#             train_data.append(im_resize)
#     return train_data
def augment_minus_image(img):
    augmented = []

    # 1. Ảnh gốc
    augmented.append(img)

    # 2. Xoay ±5 độ
    for angle in [-5, 5]:
        (h, w) = img.shape[:2]
        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
        rotated = cv2.warpAffine(img, M, (w, h), borderValue=255)
        augmented.append(rotated)

    # 4. Co chiều ngang (dấu trừ ngắn)
    h, w = img.shape
    new_w_short = int(w * 0.6)
    short = cv2.resize(img, (new_w_short, h), interpolation=cv2.INTER_AREA)
    padded_short = np.ones((h, w), dtype=np.uint8) * 255
    start_x = (w - new_w_short) // 2
    padded_short[:, start_x:start_x+new_w_short] = short
    augmented.append(padded_short)

    # # 5. Giãn chiều ngang (dấu trừ dài)
    # new_w_long = int(w * 1.4)
    # long = cv2.resize(img, (new_w_long, h), interpolation=cv2.INTER_CUBIC)
    # padded_long = np.ones((h, w), dtype=np.uint8) * 255
    # start_x = (w - new_w_long) // 2
    # if start_x < 0:  # Nếu vượt quá chiều rộng
    #     long = long[:, -start_x:start_x+w]  # Cắt lại cho đúng kích thước
    #     padded_long = long
    # else:
    #     padded_long[:, start_x:start_x+new_w_long] = long
    # augmented.append(padded_long)

    return augmented



def load_images_from_folder(folder,label):
    data = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename),cv2.IMREAD_GRAYSCALE)
        img = ~img        
        if img is not None:
            ret,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
            ctrs,ret=cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
            cnt=sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
            maxi=0
            for c in cnt:
                x,y,w,h=cv2.boundingRect(c)
                if w*h > maxi:
                    maxi = w*h
                    x_max,y_max,w_max,h_max = x,y,w,h
            im_crop = thresh[y_max:y_max+h_max+10, x_max:x_max+w_max+10]
            if label == '10':
                aug_imgs = augment_minus_image(im_crop)
                for aug in aug_imgs:
                    im_resize = cv2.resize(im_crop,(45,45))
                    im_resize = np.reshape(im_resize,(45*45))
                    data.append(im_resize)
            else:
                im_resize = cv2.resize(im_crop,(45,45))
                im_resize = np.reshape(im_resize,(45*45))
                data.append(im_resize)
    return data

In [ ]:
# data=[]
def load_and_split(folder, label):

    data = load_images_from_folder(folder,label)
    for i in range(len(data)):
        data[i] = np.append(data[i], [label])
    data = np.array(data)
    split_idx = int(0.8 * len(data))
    return data[:split_idx], data[split_idx:]

train_data, test_data = [], []

In [ ]:
for i in range(0, 10):
    folder_path = fr"C:\Users\Admin\Desktop\dl\data\extracted_images\{i}"
    train, test = load_and_split(folder_path, str(i))  # label vẫn là số nguyên
    train_data.append(train)
    test_data.append(test)


In [ ]:
# # Assign '-' = 10
# data=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\-",'10')
# len(data)
# for i in range(0,len(data)):
#     data[i]=np.append(data[i],['10'])
    
# print(len(data))

train, test = load_and_split(r"C:\Users\Admin\Desktop\dl\data\extracted_images\-", '10')
train_data.append(train)
test_data.append(test)

In [ ]:
# # Assign + = 11
# data11=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\+")

# for i in range(0,len(data11)):
#     data11[i]=np.append(data11[i],['11'])
# data=np.concatenate((data,data11))
# print(len(data))

train, test = load_and_split(r"C:\Users\Admin\Desktop\dl\data\extracted_images\+", '11')
train_data.append(train)
test_data.append(test)

In [ ]:
# data12=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\times")

# for i in range(0,len(data12)):
#     data12[i]=np.append(data12[i],['12'])
# data=np.concatenate((data,data12))
# print(len(data))

train, test = load_and_split(r"C:\Users\Admin\Desktop\dl\data\extracted_images\times", '12')
train_data.append(train)
test_data.append(test)

In [ ]:
# # data0=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\0")

# # for i in range(0,len(data0)):
# #     data0[i]=np.append(data0[i],['0'])
# # data=np.concatenate((data,data0))
# # print(len(data))

# train, test = load_and_split(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\0", '0')
# train_data.append(train)
# test_data.append(test)

In [ ]:
# # data1=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\1")

# # for i in range(0,len(data1)):
# #     data1[i]=np.append(data1[i],['1'])
# # data=np.concatenate((data,data1))
# # print(len(data))

# train, test = load_and_split(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\1", '1')
# train_data.append(train)
# test_data.append(test)

In [ ]:
# # data2=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\2")

# # for i in range(0,len(data2)):
# #     data2[i]=np.append(data2[i],['2'])
# # data=np.concatenate((data,data2))
# # print(len(data))

# train, test = load_and_split(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\2", '2')
# train_data.append(train)
# test_data.append(test)

In [ ]:
# # data3=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\3")

# # for i in range(0,len(data3)):
# #     data3[i]=np.append(data3[i],['3'])
# # data=np.concatenate((data,data3))
# # print(len(data))

# train, test = load_and_split(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\3", '3')
# train_data.append(train)
# test_data.append(test)

In [ ]:
# # data4=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\4")

# # for i in range(0,len(data4)):
# #     data4[i]=np.append(data4[i],['4'])
# # data=np.concatenate((data,data4))
# # print(len(data))

# train, test = load_and_split(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\4", '4')
# train_data.append(train)
# test_data.append(test)

In [ ]:
# data5=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\5")

# for i in range(0,len(data5)):
#     data5[i]=np.append(data5[i],['5'])
# data=np.concatenate((data,data5))
# print(len(data))

In [ ]:
# data6=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\6")

# for i in range(0,len(data6)):
#     data6[i]=np.append(data6[i],['6'])
# data=np.concatenate((data,data6))
# print(len(data))

In [ ]:
# data7=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\7")

# for i in range(0,len(data7)):
#     data7[i]=np.append(data7[i],['7'])
# data=np.concatenate((data,data7))
# print(len(data))

In [ ]:
# data8=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\8")

# for i in range(0,len(data8)):
#     data8[i]=np.append(data8[i],['8'])
# data=np.concatenate((data,data8))
# print(len(data))

In [ ]:
# data9=load_images_from_folder(r"C:\Users\Admin\Desktop\dl\handwritting_equation\data\extracted_images\9")

# for i in range(0,len(data9)):
#     data9[i]=np.append(data9[i],['9'])
# data=np.concatenate((data,data9))
# print(len(data))

In [ ]:
# df=pd.DataFrame(data,index=None)
# df.to_csv('train_final.csv',index=False)

train_data = np.concatenate(train_data)
test_data = np.concatenate(test_data)

train_data = shuffle(train_data)
test_data = shuffle(test_data)

# Xuất ra CSV
pd.DataFrame(train_data).to_csv("train_final.csv", index=False)
pd.DataFrame(test_data).to_csv("test_final.csv", index=False)

print("Tổng train:", len(train_data))
print("Tổng test:", len(test_data))

**Data Extraction has been completed**